In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='0'

import torch
from torchvision import models, transforms
from PIL import Image
import ast
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_path = '../../data/jeehoshin/allrecipe_dataset/'

In [2]:
recipes = pd.read_csv(data_path + "core-data_recipe.csv")
image_directory = data_path + "core-data-images/core-data-images"
recipe_ids = list(recipes['recipe_id'].unique())

def extract_directions(cell):
    try:
        d = ast.literal_eval(cell)
        return d.get('directions', "")
    except:
        return ""

recipes['cooking_directions'] = recipes['cooking_directions'].apply(extract_directions)

# Preprocess nutrition data (Use 7 major nutritions and format it as a string)
for i in range(len(recipes)):
    nutri = recipes.iloc[i]['nutritions']
    nutri_dict = ast.literal_eval(nutri)

    selected_keys = ['sugars', 'sodium', 'carbohydrates', 'calories', 'fat', 'saturatedFat', 'fiber']
    filtered_nutri = {key: nutri_dict[key] for key in selected_keys if key in nutri_dict}

    nutrition_summary = ""
    for key in selected_keys:
        value = filtered_nutri.get(key, {})
        amount = value.get('displayValue', 0)
        unit = value.get('unit', '')
        nutrition_summary += f"{key}: {amount}{unit}, "
    
    recipes.at[i, 'nutrition_summary'] = nutrition_summary

recipes.drop(columns=['nutritions'], inplace=True)

print(recipes.head())

   recipe_id                              recipe_name  \
0     240488        Pork Loin, Apples, and Sauerkraut   
1     218939         Foolproof Rosemary Chicken Wings   
2      87211                    Chicken Pesto Paninis   
3     245714                       Potato Bacon Pizza   
4     218545  Latin-Inspired Spicy Cream Chicken Stew   

                                           image_url  \
0  https://images.media-allrecipes.com/userphotos...   
1  https://images.media-allrecipes.com/userphotos...   
2  https://images.media-allrecipes.com/userphotos...   
3  https://images.media-allrecipes.com/userphotos...   
4  https://images.media-allrecipes.com/userphotos...   

                                         ingredients  \
0  sauerkraut drained^Granny Smith apples sliced^...   
1  chicken wings^sprigs rosemary^head garlic^oliv...   
2  focaccia bread quartered^prepared basil pesto^...   
3  red potatoes^strips bacon^Sauce:^heavy whippin...   
4  skinless boneless chicken breast halv

In [ ]:
def process_row_detailed(row):
    return pd.Series({
        'id': row['recipe_id'],
        'name': row['recipe_name'],
        'ingredients': row['ingredients'],
        'nutrition': row['nutrition_summary'],
        'steps' : row['cooking_directions']
    })

detailed_summary_recipe = recipes.apply(process_row_detailed, axis=1)
print(detailed_summary_recipe.columns)
print(detailed_summary_recipe.head(3))

detailed_summary_recipe.to_csv(data_path + 'recipe_textual_metadata.csv', index=False)

Index(['id', 'name', 'ingredients', 'nutrition', 'steps'], dtype='object')
       id                               name  \
0  240488  Pork Loin, Apples, and Sauerkraut   
1  218939   Foolproof Rosemary Chicken Wings   
2   87211              Chicken Pesto Paninis   

                                         ingredients  \
0  sauerkraut drained^Granny Smith apples sliced^...   
1  chicken wings^sprigs rosemary^head garlic^oliv...   
2  focaccia bread quartered^prepared basil pesto^...   

                                           nutrition  \
0  sugars: 19.8g, sodium: 2607mg, carbohydrates: ...   
1  sugars: 0.2g, sodium: 763mg, carbohydrates: 6....   
2  sugars: 2g, sodium: 1076mg, carbohydrates: 60....   

                                               steps  
0  Prep\n15 m\nCook\n2 h 30 m\nReady In\n2 h 45 m...  
1  Prep\n20 m\nCook\n40 m\nReady In\n1 h\nPreheat...  
2  Prep\n15 m\nCook\n5 m\nReady In\n20 m\nPreheat...  


In [ ]:
# =========================
# [Visual Feature] CaffeNet
# =========================
try:
    from torchvision.models import AlexNet_Weights
    alexnet_weights = AlexNet_Weights.DEFAULT
except Exception:
    alexnet_weights = 'DEFAULT'

caffenet = models.alexnet(weights=alexnet_weights)
# Trim the last classification layer to get a 4096-D embedding
caffenet.classifier = torch.nn.Sequential(*list(caffenet.classifier.children())[:-1])
caffenet.eval()
caffenet.to(device)

# Use the canonical ImageNet preprocessing if available; otherwise fall back to a manual transform
try:
    transform = alexnet_weights.transforms()
except Exception:
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

batch_size = 32
image_batch = []
id_batch = []
visual_features = []

for rid in tqdm(recipe_ids, desc="visual_encoding"):
    image_path = os.path.join(image_directory, f"{rid}.jpg")
    if not os.path.exists(image_path):
        continue
    try:
        img = Image.open(image_path).convert("RGB")
        img_tensor = transform(img)
        image_batch.append(img_tensor)
        id_batch.append(rid)
    except Exception:
        continue

    if len(image_batch) == batch_size:
        batch_tensor = torch.stack(image_batch).to(device)
        with torch.no_grad():
            batch_features = caffenet(batch_tensor).cpu().numpy()  # (N, 4096)
        for r_id, feat in zip(id_batch, batch_features):
            visual_features.append({'recipe_id': r_id, 'visual_feature': feat})
        image_batch, id_batch = [], []

# Handle leftover images
if len(image_batch) > 0:
    batch_tensor = torch.stack(image_batch).to(device)
    with torch.no_grad():
        batch_features = caffenet(batch_tensor).cpu().numpy()
    for r_id, feat in zip(id_batch, batch_features):
        visual_features.append({'recipe_id': r_id, 'visual_feature': feat})

visual_df = pd.DataFrame(visual_features)
print(visual_df.head())

# ===========================================
# [Textual Feature] (unchanged, per request)
# ===========================================
recipes['recipe_name'] = recipes['recipe_name'].fillna('')
recipes['ingredients'] = recipes['ingredients'].fillna('')
recipes['cooking_directions'] = recipes['cooking_directions'].fillna('')

recipes['description'] = (
    recipes['recipe_name'] + ' ' +
    recipes['ingredients'] + ' ' +
    recipes['cooking_directions'] + ' ' + 
    recipes['nutrition_summary']
)

recipes.drop(columns=['recipe_name', 'image_url', 'ingredients', 'cooking_directions', 'nutrition_summary'], inplace=True)

model = SentenceTransformer('all-MiniLM-L6-v2', device = device)
descriptions = recipes['description'].tolist()
recipe_ids_text = recipes['recipe_id'].tolist()

# Encode with batching to avoid OOM
text_embeddings = model.encode(descriptions, batch_size=32, show_progress_bar=True)

textual_df = pd.DataFrame({
    'recipe_id': recipe_ids_text,
    'textual_feature': list(text_embeddings)
})

print(textual_df.head())

visual_encoding: 100%|███████████████| 45630/45630 [09:56<00:00, 76.44it/s]


   recipe_id                                     visual_feature
0     240488  [0.0, 0.0, 0.0, 0.0, 0.8951623, 6.2409043, 0.0...
1     218939  [0.0, 0.0, 0.37667835, 0.0, 1.3771789, 0.0, 0....
2      87211  [0.0, 0.0, 0.0, 0.0, 0.10161859, 0.0, 0.0, 0.0...
3     245714  [0.0, 0.0, 0.0, 0.0, 2.0885088, 6.285, 0.0, 0....
4     218545  [0.0, 0.0, 0.0, 0.0, 2.1782794, 0.0, 0.0, 0.0,...


Batches:   0%|          | 0/1426 [00:00<?, ?it/s]

   recipe_id                                    textual_feature
0     240488  [0.0031801027, 0.03284959, -0.048994727, -0.01...
1     218939  [-0.03752384, -0.05383142, -0.055957776, -0.02...
2      87211  [-0.059003763, -0.016003406, -0.04236307, -0.0...
3     245714  [-0.04850639, -0.0357542, -0.039720565, -0.048...
4     218545  [-0.038224142, -0.09296297, -0.041824777, 0.01...


In [5]:
print(len(visual_df.iloc[0]['visual_feature']))
print(len(textual_df.iloc[0]['textual_feature']))

4096
384


In [6]:
# load user ID mapping & item ID mapping
user_id_mapping = pd.read_csv(data_path + 'u_id_mapping.csv', sep='\t')
item_id_mapping = pd.read_csv(data_path + 'i_id_mapping.csv', sep='\t')

visual_dict = dict(zip(visual_df['recipe_id'], visual_df['visual_feature']))
textual_dict = dict(zip(textual_df['recipe_id'], textual_df['textual_feature']))

# === Create visual feature array ===
feat_dim_v = len(next(iter(visual_dict.values())))  # e.g., 4096
print(feat_dim_v)
visual_feat = np.zeros((len(recipe_ids), feat_dim_v))

for idx, row in item_id_mapping.iterrows():
    rid = row['recipe_id']
    visual_feat[idx] = visual_dict.get(rid, np.zeros(feat_dim_v))  # fallback if missing

np.save(data_path + 'image_feat.npy', visual_feat)
print('image_feat.npy created')

# === Create textual feature array ===
feat_dim_t = len(next(iter(textual_dict.values())))  # e.g., 384
print(feat_dim_t)
textual_feat = np.zeros((len(recipe_ids), feat_dim_t))

for idx, row in item_id_mapping.iterrows():
    rid = row['recipe_id']
    textual_feat[idx] = textual_dict.get(rid, np.zeros(feat_dim_t))  # fallback if missing

np.save(data_path + 'text_feat.npy', textual_feat)
print('text_feat.npy created')

4096
image_feat.npy created
384
text_feat.npy created


In [ ]:
with open(data_path + 'allrecipe.inter', 'r', encoding='utf-8') as f:
    for _ in range(10):
        print(f.readline().strip())

i_mapping = pd.read_csv(data_path + 'i_id_mapping.csv')
u_mapping = pd.read_csv(data_path + 'u_id_mapping.csv')

print("\nItem ID Mapping:")
print(i_mapping.head())

print("\nUser ID Mapping:")
print(u_mapping.head())

image_feat = np.load(data_path + 'image_feat.npy')
text_feat = np.load(data_path + 'text_feat.npy')

print("Image feature shape:", image_feat.shape)
print(image_feat[:3])  # show first 3 entries
print(image_feat[0][:10])
print("\nText feature shape:", text_feat.shape)
print(text_feat[:3])

userID	itemID	rating	timestamp	x_label
0	0	5	"2010-08-25T14:38:53.84
"	0
0	1	4	"2010-09-09T14:04:45.733
"	0
0	2	5	"2010-08-16T14:51:25.833
"	0
1	3	4	"2009-03-15T12:10:20.85
"	0
2	4	5	"2005-10-04T15:43:36.653

Item ID Mapping:
  recipe_id\titemID
0          17991\t0
1         170724\t1
2          18045\t2
3          60598\t3
4          47519\t4

User ID Mapping:
  user_id\tuserID
0      5215572\t0
1      3622615\t1
2      1313770\t2
3      3181149\t3
4       880574\t4
Image feature shape: (45630, 4096)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0.        0.        0.        0.        0.6379146 0.        0.
 0.        0.        0.       ]

Text feature shape: (45630, 384)
[[-0.10659684 -0.04303089 -0.00518711 ...  0.06801672 -0.00361427
   0.00287551]
 [ 0.01234793 -0.05021437 -0.07693625 ... -0.05673526  0.0030512
  -0.02264609]
 [ 0.02027458 -0.03360106 -0.02195448 ...  0.01065957  0.00821418
  -0.03242688]]
